# Daten erkunden


Python Pakete einbinden.

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import matplotlib.pylab as plt

# Lageparameter in der deskriptiven Statistik

Lageparameter geben an, wo sich der größte Teil einer Stichprobe befindet. Sie werden auch als Maß der zentralen Tendenz bezeichnet.

## Beispiel: Lageparameter zu Bevölkerung und Verbrechen in den USA

Laden wir die Daten in einen pandas DataFrame und lassen uns den Inhalt anzeigen.

In [ ]:
state = pd.read_csv('state.csv')
state.head(8)

### Wie groß ist Bevölkerungsanzahl in den Staaten der USA im Mittel?

Gängige Lageparameter sind um Antworten auf diese Frage zu erhalten sind:
* das arthmetische Mittel (= Durchschnitt)
* das getrimmte arithmetische Mittel 
* der Median (= der mittlere Wert)

`mean` und `median` können mit den pandas Methoden für DataFrames berechnet werden.. 

Der getrimmt Mittlwert erfordert die `trim_mean` Funktion in _scipy.stats_.

In [ ]:
state['Population'].mean()

Ein Bundesstaat hat also im Mittel ca. 6 Mio Einwohner. Lassen wir uns zusätzlich die Extremwerte anzeigen.

In [ ]:
state['Population'].max()

In [ ]:
state['Population'].min()

Welcher Staat hat die kleinste Bevölkerungsanzahl?

In [ ]:
state['Population'].idxmin()

In [ ]:
state.iloc[49,:]

In [ ]:
state.iloc[state['Population'].idxmin()]

Diese Ausreißer kann man z.B. mit den getrimmten/trunkierten Mittelwert herausfiltern. Im Beispiel unten werden die oberen und unteren 10% der Werte ignoriert.

In [ ]:
trim_mean(state['Population'], 0.1)

Dies liegt viel näher an einem weitere wichtigen Lageparameter: dem Median. hierbei wird die Reihe sortiert (hier nach Bevölkerungsanzahl) und der Wert selektiert, der genau in der Mitte liegt. 

In [ ]:
state['Population'].median()

Um Mittelwerte für Skalen mit abhängigen Bezugsgrößen zu ermitteln kann der sogenannte gewichtete Mittelwert verwendet werden. Der gewichtete Mittelwert is im Paket numpy implementiert. Für den gewichteten Median können wir das spezialisierte paket `wquantiles` verwenden (https://pypi.org/project/wquantiles/).

In [ ]:
state['Murder.Rate'].mean()


In [ ]:
state['Murder.Rate'].median()


In [ ]:
np.average(state['Murder.Rate'], weights=state['Population'])

In unserem Beispiel ergibt sich keine große Änderung da die Mordrate schon auf Morde pro 100.000 Einwohner bezogen ermittelt wird. Wäre stattdessen die Anzahl der Morde als absoluter Wert im data set, würde nur der gewichtete Mittelwert eine korrekte Analyse ergeben.

Auch der Median kann mit Hilfe von Gewichten beeinflusst werden. Im Beispiel unten, benutzen wir dazu das Paket wquantiles. 

In [ ]:
wquantiles.median(state['Murder.Rate'], weights=state['Population'])

# Streuungsparameter

In [ ]:
state.head(8)

## Die Standardabweichung

Dieses Maß beschreibt, wie weit die gemessenen Werte im Schnitt vom Durchschnitt (genauer: vom arithmetisches Mittel) abweichen (sic!). 

D.h. für jeden Wert wird die Differenz zum Mittelwert gebildet. Diese Differenzen werden quadriert (um zu vermeiden das sich positive und negative Differenzen ausgleichen) und aufsummiert und anschließend durch die Anzahl der Messungen -1 geteilt. Als Ergebnis erhalten wir die sogenannte Varianz. Die Quadratwurzel aus diesem Wert ist die Standardabweichung.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/96319e4ec5a4317a943c96b8a6408e71923fcc2c)

Für unser Beispiel bedeutet dies, dass die Mordrate Bundestaaten im Mittel um 1.9 Punkte vom Durchschnitt abweicht.

In [ ]:
state['Murder.Rate'].std()

Der Interquartilabstand IQA (bzw. IQR) wird als Differenz zwischen den 75% und 25% Quantil berechnet. Das Ergebnis beschreibt die Breite des Korridors, in dem die mittleren 50% der Stichprobe liegen.  

In [ ]:
state['Population'].quantile(0.75) - state['Population'].quantile(0.25)

Eine weitere Methode ist die Berechnung der mittleren absoluten Abweichung vom Median (auch MAD = median absolute deviation). Dazu können wir eine Funktion des _statsmodels_ pakets benutzen.

In [ ]:
robust.scale.mad(state['Population'])
abs(state['Population'] - state['Population'].median()).median() / 0.6744897501960817

In [ ]:
robust.scale.mad(state['Murder.Rate'])
abs(state['Murder.Rate'] - state['Murder.Rate'].median()).median() / 0.6744897501960817

## Perzentile and Boxplots
_Pandas_ bietet die Methode `quantile` für DataFrames an. Damit können flexibel verschiedenste Quantile ermittelt werden.

In [ ]:
state['Murder.Rate'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:

percentages = [0.05, 0.25, 0.5, 0.75, 0.95]
df = pd.DataFrame(state['Murder.Rate'].quantile(percentages))
df.index = [f'{p * 100}%' for p in percentages]
df.transpose()

_Pandas_ bietet ein Auswahl an Visualisierungen für die explorative Analyse, z.B. Boxplots.

Ein Boxplot hat eine Box, die sich vom 25% bis zum 75 % Quantil erstreckt. 

Die Enden (auch Antennen oder Whisker gennant) ober- und unterhalb der Box repräsentieren das Maximum und das Minimum der Stichprobe, der Strich in der Mitte der Box zeigt den Median.

Werte die mehr als das 1,5-fache des IQA von der Box abweichen werden als Ausreißer mit Kreisen gekennzeichnet. D.h. der letzte Wert der Whisker ist der, der noch kleiner als 1,5 * IQA ist.

Werte die mehr als das 3-fache des IQA betragen werden als Extremwerte bezeichnet und i.d.R. mit "x" gekennzeichnet. Beim Standard Boxplot von _pandas_ ist dies jedoch nicht vorgesehen.

In [ ]:
ax = (state['Population']/1_000_000).plot.box(figsize=(7, 8))
ax.set_ylabel('Population (millions)')

plt.tight_layout()
plt.show()

## Häufigkeitsverteilungen und Histogramme
Die Methode `cut` in _pandas_ teilt Daten in Klassen(="Töpfe") auf. 

Der folgende Code erzeugt 10 gleich große Klassen aus den Bevölkerungsdaten der Staaten. Die Methode `value_counts` zeigt die Häufigkeiten für die einzelnen Klassen.

In [ ]:
binnedPopulation = pd.cut(state['Population'], 10)

In [ ]:
binnedPopulation

In [ ]:

binnedPopulation.value_counts()

Eine etwas komplexe aber gut funktionierende Methode, diese Häufigkeitsverteilung mit den ursprünglichen Daten aus dem state DataFrame zu verknüpfen sehen Sie unten. Zuerst wird die Series binnedPopulation mit dem bisherigen DataFrame state per `concat()` verknüpft. 

In [ ]:
binnedPopulation.name = 'binnedPopulation'
df = pd.concat([state, binnedPopulation], axis=1)
df = df.sort_values(by='Population')

Anschließend wird über das Ergebnis (also die Gruppen) eines `groupby()`iteriert, um die einzelnen Abkürzungen der Staaten in der Spalte "States" zusammenzuführen. In "BinRange" sind die Gruppenbezeichner, in "Count" die Anzahl der Staaten in jeder Gruppe bzw. Klasse dargestellt.

In [ ]:
groups = []
for group, subset in df.groupby(by='binnedPopulation'):
    groups.append({
        'BinRange': group,
        'Count': len(subset),
        'States': ','.join(subset.Abbreviation)
    })
pd.DataFrame(groups)

_Pandas_ unterstützt auch die schnelle Erzeugung von Histogrammen. So kann schnell eine Häufigkeitsverteilung in einer Stichprobe sichtbargemacht werden. Per Standard werden zehn KLassen gebildet, dies lässt sich aber über die Parameter von `hist()` steuern.

In [ ]:
ax = (state['Population'] / 1_000_000).plot.hist(figsize=(8, 8))
ax.set_xlabel('Population (millions)')

plt.tight_layout()
plt.show()

## Dichteschätzer
Der Einsatz eines (Kern-)dichteschätzers ermöglicht eine genauere weil stetige Einschätzung der Verteilung der der Daten. Der Parameter `bw_method` steuert die Glättung der Kurve.

In [ ]:
ax = state['Murder.Rate'].plot.hist(density=True, xlim=[0, 12], 
                                    bins=range(1,12), figsize=(8, 8))
state['Murder.Rate'].plot.density(ax=ax)
ax.set_xlabel('Murder Rate (per 100,000)')

plt.tight_layout()
plt.show()